# 基础概念
* 推理：数据通过已经训练好的模型得到输出结果这一过程。
* 推理系统：一个完整的软件框架或者平台，包含推理引擎、数据处理、模型管理、资源调度和API接口等模块
* 推理引擎：专门负责执行模型计算，直接加载训练好的模型，根据输入数据生成推理结果。目前市场上已有多种推理引擎，如字节跳动的 LightSeq、Meta AI 的 AITemplate、英伟达的 TensorRT，以及华为的 MindSpore Lite 和腾讯的 NCNN 等
* 低比特量化：用更低的比特表示模型的权重值和激活值，例如用8位bit
* 剪枝：移除网络中的冗余连接或者神经元减少模型的大小。或者通过设置阈值删除权重较小的连接或神经元实现。
* 联合学习：在不共享本地数据的前提下，通过一个核心服务器和本地服务器协同训练一个全局共享的机器学习模型。步骤为：各参与方在本地训练模型，加密上传模型参数，全局服务器整合所有参数，更新全局模型，将全局模型在返回给参与方。
* ONNX(Open Neural Network Exchange)是一种通用的神经网络模型**表示格式**，旨在解决不同框架之间模型不兼容的问题，实现不同框架下模型的相互转换。

# 推理系统
## AI 生命周期
整个 AI 生命周期包含数据准备、模型训练和推理、部署优化三个核心部分。
### 数据准备
1. 存储问题：
   1. 可以将所有的数据存储在一个中心服务器上，模型在这个中心服务器上训练。
   2. 数据分散存储在不同的服务器上，通过联合学习的方式进行模型训练，去中心化，分布式训练。
2. 数据处理：
   1. 对输入的数据在每一维度进行归一化处理，避免某一维数值分布太大导致其占主导地位。
3. 数据增强

### 训练和推理
* 训练更需要使用大批此的数据，避免归一化偏离数据分布，提升训练过程的稳定性和效率。同时需要大量的梯度计算，因此需要更高的计算库和 AI 框架。
* 推理阶段不涉及梯度和计算图，更关注性能和效率，因此需要关注模型的计算速度和内存占用，方便实时和高吞吐量的处理输入数据

### 部署
1. 通过剪枝、量化、知识蒸馏等方法对模型进行优化和压缩
2. 选择合适的推理引擎，常用的推理引擎如 TensorRT、OpenVINO、ONNX Runtime 等针对不同硬件设备进行优化，提供高效的模型推理能力。有时需要将模型从训练框架转换为推理引擎支持的格式
3. 创建 API 接口、配置服务器、设置数据传输和存储等。

## 推理相较于训练的挑战
训练通常需要支持大批量计算的中心服务器。推理要求服务器24时在线，并且能够处理大的并发量，提供高效稳定的推理服务。所有挑战如下：
1. 长期运行服务的要求，此外还像传统互联网服务一下，低延迟，高并发
2. 资源约束：例如在一些小的设备上，需要更小的内存，更低的功率等。
3. 不需要反向传播和梯度下降：可以适量的使用一些牺牲数据精度的策略，例如量化、稀疏性等
4. 设备型号多样性

## 模型推理常见步骤
以 Pytorch 实现的 ResNet50 模型在 TensorRT 的推理过程为例

In [ ]:
import torch
import torchvision.models as models
import tensorrt as trt
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit

# 步骤 1：加载 PyTorch 模型并转换为 ONNX 格式
model = models.resnet50(pretrained=True) # 加载预训练的 ResNet50 模型
model.eval()
dummy_input = torch.randn(1, 3, 224, 224) # 创建一个示例输入
torch.onnx.export(model, dummy_input, "resnet50.onnx", opset_version=11) # 将模型导出为 ONNX 格式

# 步骤 2：使用 TensorRT 将 ONNX 模型转换为 TensorRT 引擎
TRT_LOGGER = trt.Logger(trt.Logger.WARNING) # 创建一个 Logger
EXPLICIT_BATCH = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH) # 如果是动态输入，需要显式指定 EXPLICIT_BATCH
with trt.Builder(TRT_LOGGER) as builder, builder.create_network(EXPLICIT_BATCH) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
    # 创建一个 Builder 和 Network
    # builder 创建计算图 INetworkDefinition
    builder.max_workspace_size = 1 << 30  # 1GB ICudaEngine 执行时 GPU 最大需要的空间
    builder.max_batch_size = 1 # 执行时最大可以使用的 batchsize

    with open("resnet50.onnx", "rb") as model_file:
        parser.parse(model_file.read())  # 解析 ONNX 文件

    engine = builder.build_cuda_engine(network)  # 构建 TensorRT 引擎

    with open("resnet50.trt", "wb") as f:
        # 将引擎保存到文件
        f.write(engine.serialize())

# 步骤 3：使用 TensorRT 引擎进行推理
def load_engine(engine_file_path):
    # 加载 TensorRT 引擎
    with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

engine = load_engine("resnet50.trt")
context = engine.create_execution_context() # 将引擎应用到不同的 GPU 上配置执行环境

# 准备输入和输出缓冲区
input_shape = (1, 3, 224, 224)
output_shape = (1, 1000)
input_size = trt.volume(input_shape) * trt.float32.itemsize
output_size = trt.volume(output_shape) * trt.float32.itemsize
d_input = cuda.mem_alloc(input_size)
d_output = cuda.mem_alloc(output_size)
stream = cuda.Stream() # 创建流
input_data = np.random.random(input_shape).astype(np.float32)# 创建输入数据
cuda.memcpy_htod_async(d_input, input_data, stream) # 复制输入数据到 GPU

# 推理
context.execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
# 从 GPU 复制输出数据
output_data = np.empty(output_shape, dtype=np.float32)
cuda.memcpy_dtoh_async(output_data, d_output, stream) # 获取推理结果，并将结果拷贝到主存
stream.synchronize() # 同步流
print("Predicted output:", output_data)
